
# Semantic Kernel - The Kernel 

Semantic Kernel is an open-source SDK designed for developers to easily integrate AI agents into their existing systems, facilitating automation and enhancing user experiences. This SDK bridges the gap between AI capabilities and practical application, making it straightforward to bring AI insights into actionable processes.




## Core Features
- AI Service Integration: 
It offers a uniform interface for connecting with various AI services like OpenAI, Azure OpenAI, HuggingFace, and Dall-e. This ensures developers can easily integrate different AI technologies without worrying about compatibility issues.

- Extending AI to Real-World Actions: 
Beyond mere integration, the SDK allows for the execution of native code and actions based on AI outputs, supporting complex workflows through input and output chaining.Unified Interface to AI services
Extend AI ablility to exteral world


## Kernel
Kernel is the center component of an AI applciation that manages all AI resources.
With regiestered AI models, services (logging, telemetry) and plugings, 
the kernel will handle all the execution details: 

1. Select the best AI service to run the prompt.

2. Build the prompt using the provided prompt template.

3. Send the prompt to the AI service.

4. Receive and parse the response.

5. Pipeline, chain all executions


## Example: A simple agent

In the following code, we will demonstrate the minimum steps for building an AI agent that follows user's instructions to complete tasks on a local system.

1. register an AI service. In this example, it is an Azure OpenAI "GPT-4" deployment.

2. add two plugings: TimePlugin and LightPlugin.

 - [TimePlugin](https://learn.microsoft.com/en-us/semantic-kernel/agents/plugins/out-of-the-box-plugins?tabs=Csharp) is an out-of-the-box plugin provided by Semantic Kernel, 
 
 - [LightPlugin]((./plugins/LightPlugin.cs).) is a plugin written in native code. 
 
    This plugin contains group of function that controls a dummy lighting system. Check its implementation code [here](./plugins/LightPlugin.cs)
 
Under the hood, Semantic Kernel handles all the heavy lifting by constructing requests to the AI service, parsing the responses, invoking local functions, and chaining the messages.

### Step 1: Importing the required libraries

In [3]:
#r "nuget: Microsoft.SemanticKernel, 1.3.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.3.1-alpha"
#!import config/Settings.cs 
#!import lib/Usings.cs
#!import lib/Utilities.cs
#!import plugins/LightPlugin.cs

Installed Packages Microsoft.SemanticKernel, 1.3.1 Microsoft.SemanticKernel.Plugins.Core, 1.3.1-alpha

### Step 2: build the kernel

In [4]:
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
var builder = Kernel.CreateBuilder();

//register ai serivces
builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);

//register plugins
#pragma warning disable SKEXP0050
builder.Plugins.AddFromType<TimePlugin>();
builder.Plugins.AddFromType<LightPlugin>();

var kernel = builder.Build();


### Step 3: send a request to the agent. 

We have AI service and plugins configured in prevous step.

Now lets make a request to that agent.

In [14]:
var input = "Turn on the lights if it is night time.";

Console.WriteLine($"[User] > {input}");

ChatHistory history = [];
history.AddUserMessage(input);

// Get chat completion service
var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

// Enable auto function calling
OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new()
{
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

// Get the response from the AI
var result = await chatCompletionService.GetChatMessageContentAsync(
        history,
        executionSettings: openAIPromptExecutionSettings,
        kernel: kernel);

// Add the message from the agent to the chat history
history.AddMessage(result.Role, result.Content);

Console.WriteLine("[Assistant] > " + result);


[User] > Turn on the lights if it is night time.
[Assistant] > The lights are already on, and it is currently evening. There's no need to change anything. Is there anything else I can help with?


If everything is set up correctly, we should see the output tells us the dummy lighs have been turned on/off. 

## What has happened under the hood

<font >

Despite receiving only one user input, the kernel efficiently coordinated a series of task executions.

First, it selected the default AI service to transmit the prompt. Alongside the prompt text, it included function metadata to inform the AI service about available functions.

Subsequently, it parsed the AI service response to initiate function calls and chain executions seamlessly.

</font> 

Let's closely examine the chat history containing all intermediate message exchanges between the AI service and the kerne"l.

In [15]:
history.Display()

[ Turn on the lights if it is night time., , 17, on, The lights are already on, and it is currently evening. There's no need to change anything. Is there anything else I can help with? ] Count 5 (values) index type value 0 Microsoft.SemanticKernel.ChatMessageContent Turn on the lights if it is night time. Role user Label user Content Turn on the lights if it is night time. Items <null> Encoding System.Text.UTF8Encoding+UTF8EncodingSealed Preamble System.Text.UTF8Encoding+UTF8EncodingSealed BodyName utf-8 EncodingName Unicode (UTF-8) HeaderName utf-8 WebName utf-8 WindowsCodePage 1200 IsBrowserDisplay True IsBrowserSave True IsMailNewsDisplay True IsMailNewsSave True IsSingleByte False EncoderFallback System.Text.EncoderReplacementFallback DefaultString � MaxCharCount 1 DecoderFallback System.Text.DecoderReplacementFallback DefaultString � MaxCharCount 1 IsReadOnly True CodePage 65001 InnerContent <null> ModelId <null> Metadata <null> 1 Microsoft.SemanticKernel.Connectors.OpenAI.OpenAIChatMessageContent ToolCalls index value 0 Azure.AI.OpenAI.ChatCompletionsFunctionToolCall Name TimePlugin-HourNumber Arguments {} Id call_DaRoK1Rttu9QHMvKHFchopWI 1 Azure.AI.OpenAI.ChatCompletionsFunctionToolCall Name LightPlugin-GetState Arguments {} Id call_MKcv5IwzqhFqWzJYejsx1CWM Role assistant Label assistant Content <null> Items <null> Encoding System.Text.UTF8Encoding+UTF8EncodingSealed Preamble System.Text.UTF8Encoding+UTF8EncodingSealed BodyName utf-8 EncodingName Unicode (UTF-8) HeaderName utf-8 WebName utf-8 WindowsCodePage 1200 IsBrowserDisplay True IsBrowserSave True IsMailNewsDisplay True IsMailNewsSave True IsSingleByte False EncoderFallback System.Text.EncoderReplacementFallback DefaultString � MaxCharCount 1 DecoderFallback System.Text.DecoderReplacementFallback DefaultString � MaxCharCount 1 IsReadOnly True CodePage 65001 InnerContent Azure.AI.OpenAI.ChatResponseMessage Role assistant Content <null> ToolCalls index value 0 Azure.AI.OpenAI.ChatCompletionsFunctionToolCall Name TimePlugin-HourNumber Arguments {} Id call_DaRoK1Rttu9QHMvKHFchopWI 1 Azure.AI.OpenAI.ChatCompletionsFunctionToolCall Name LightPlugin-GetState Arguments {} Id call_MKcv5IwzqhFqWzJYejsx1CWM FunctionCall <null> AzureExtensionsContext <null> ModelId GPT-4 Metadata key type value Id System.String chatcmpl-8wHiR7EW0r4GRLhaqpbxXkP4MNyLU Created System.DateTimeOffset 2024-02-25 22:56:55Z PromptFilterResults System.Collections.Generic.List<Azure.AI.OpenAI.ContentFilterResultsForPrompt> index value 0 Azure.AI.OpenAI.ContentFilterResultsForPrompt PromptIndex 0 ContentFilterResults Azure.AI.OpenAI.ContentFilterResultDetailsForPrompt Sexual Azure.AI.OpenAI.ContentFilterResult Violence Azure.AI.OpenAI.ContentFilterResult Hate Azure.AI.OpenAI.ContentFilterResult SelfHarm Azure.AI.OpenAI.ContentFilterResult Profanity <null> CustomBlocklists [ ] Error <null> Jailbreak <null> SystemFingerprint System.String fp_8abb16fa4e Usage Azure.AI.OpenAI.CompletionsUsage Azure.AI.OpenAI.CompletionsUsage CompletionTokens 44 PromptTokens 522 TotalTokens 566 ChatResponseMessage.FunctionToolCalls System.Collections.Generic.List<Azure.AI.OpenAI.ChatCompletionsFunctionToolCall> index value 0 Azure.AI.OpenAI.ChatCompletionsFunctionToolCall Name TimePlugin-HourNumber Arguments {} Id call_DaRoK1Rttu9QHMvKHFchopWI 1 Azure.AI.OpenAI.ChatCompletionsFunctionToolCall Name LightPlugin-GetState Arguments {} Id call_MKcv5IwzqhFqWzJYejsx1CWM 2 Microsoft.SemanticKernel.ChatMessageContent 17 Role tool Label tool Content 17 Items <null> Encoding System.Text.UTF8Encoding+UTF8EncodingSealed Preamble System.Text.UTF8Encoding+UTF8EncodingSealed BodyName utf-8 EncodingName Unicode (UTF-8) HeaderName utf-8 WebName utf-8 WindowsCodePage 1200 IsBrowserDisplay True IsBrowserSave True IsMailNewsDisplay True IsMailNewsSave True IsSingleByte False EncoderFallback System.Text.EncoderReplacementFallback DefaultString � MaxCharCount 1 DecoderFallback System.Text.DecoderReplacementFallback DefaultString � MaxCharCount 1 I

### We can ask the AI service to explain its thought process.

In [ ]:
history.AddUserMessage("explain your thought process");
result = await chatCompletionService.GetChatMessageContentAsync(
        history,
        executionSettings: openAIPromptExecutionSettings,
        kernel: kernel);

// Add the message from the agent to the chat history
history.AddMessage(result.Role, result.Content);

result.Display()

## Wrapping up

  Semantic Kernel abstracts away complex AI service integration logic and provides a uniform interface to communicate with different AI providers.

  With its plugin system, we can extend AI cabilities by calling external tools.

  Semantic Kernel also manages the history and orchestrates function executions.